# Prepare Evaluation Framework

In [1]:
# make sure that the evaluation framework directory is in PATH
import sys
sys.path.append('./evaluation_framework/')

# initialize framework manager
from evaluation_framework.manager import FrameworkManager
evaluation_manager = FrameworkManager()

Start evaluation...


# Run Evaluation Framework with Individual Embedding Files

In [ ]:
# run all approaches (may take several hours per approach!)
embedding_approaches = ['rdf2vec', 'DeepWalk', 'Node2Vec', 'KGlove', 'rdf2vecOA', 'TransE-L1', 'TransE-L2', 'TransR', 'RotatE', 'ComplEx', 'DistMult', 'RESCAL']
for approach in embedding_approaches:
    path_to_embedding_file = f'./data/vectors_dbpedia_{approach}.txt'
    evaluation_manager.evaluate(path_to_embedding_file, vector_size=200, parallel=True, debugging_mode=False)

# Visualize Results

In [ ]:
# The Evaluation Framework generates a file with the name `comparison.csv` containing all the results.
# We load this file with pandas and display the relevant results here

import pandas as pd

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 199)

# map the names of the approaches to something more readable
datasets = {
    'data/vectors_dbpedia_rdf2vec_200_cosine_2_1': 'Rdf2Vec',
    'data/vectors_dbpedia_DeepWalk_200_cosine_2_1': 'DeepWalk',
    'data/vectors_dbpedia_Node2Vec_200_cosine_2_1': 'Node2Vec',
    'data/vectors_dbpedia_KGlove_200_cosine_2_1': 'KGlove',
    'data/vectors_dbpedia_rdf2vecOA_200_cosine_2_1': 'Rdf2Vec_oa',
    'data/vectors_dbpedia_TransE-L1_200_cosine_2_1': 'TransE-L1',
    'data/vectors_dbpedia_TransE-L2_200_cosine_2_1': 'TransE-L2',
    'data/vectors_dbpedia_TransR_200_cosine_2_1': 'TransR',
    'data/vectors_dbpedia_RotatE_200_cosine_2_1': 'RotatE',
    'data/vectors_dbpedia_DistMult_200_cosine_2_1': 'DistMult',
    'data/vectors_dbpedia_RESCAL_200_cosine_2_1': 'RESCAL',
    'data/vectors_dbpedia_ComplEx_200_cosine_2_1': 'ComplEx',
}

# load comparison.csv file
df = pd.read_csv('./comparison.csv', sep=' ')

# compute best results per gold-standard-file and select relevant metrics
dfbest = df[(df['metric'].isin([
    'root_mean_squared_error',  # Regression
    'harmonic_mean',  # Document Similarity
    'clustering_accuracy',  # Clustering
    'kendalltau_correlation',  # Entity Relatedness
    'accuracy'  # Classification, Semantic Analogies
])) & (df['model'] != 'with_weights')]  # ignore weights
pd.concat([
    # use minimum for regression and maximum for the rest
    dfbest[dfbest['task_name'] != 'Regression'].pivot_table(values='score_value', columns=['test_name'], index=['task_name', 'gold_standard_file', 'metric'], aggfunc='max').filter(items=list(datasets), axis='columns').rename(columns=datasets).dropna(how='all').fillna(0),
    dfbest[dfbest['task_name'] == 'Regression'].pivot_table(values='score_value', columns=['test_name'], index=['task_name', 'gold_standard_file', 'metric'], aggfunc='min').filter(items=list(datasets), axis='columns').rename(columns=datasets).dropna(how='all').fillna(0)
]).rename(index={'accuracy': 'ACC', 'clustering_accuracy': 'ACC', 'root_mean_squared_error': 'RMSE', 'kendalltau_correlation': 'KendallTau', 'harmonic_mean': 'HarmonicMean'}).round(decimals=4)